In [6]:
import awswrangler as wr
import pandas as pd
import numpy as np
import boto3
import os

s3 = boto3.client('s3')
os.environ['snsARN'] = 'arn:aws:sns:ap-northeast-2:630190875472:temp_marketstatus-lambda-error-topic'

In [18]:
import awswrangler as wr
import pandas as pd
import numpy as np
import boto3

df = wr.s3.read_csv("s3://marketstatus-metas/master/")
sess = boto3.Session(region_name='us-east-1') 

## Change Data Type
df = df[['Category', 'Importer', 'Brand', 'Model', 'Model (until 2021)', 'Model group', 'Body size',
         'Body style', 'Fuel', 'Brand segment', 'CC', "Price('000)", 'LCV', 'Commercial Type']].copy()
for i in list(df):
    df[i] = df[i].str.replace('  ', ' ')
    df[i] = df[i].str.rstrip()
    df[i] = df[i].str.lstrip()
df.fillna('-', inplace = True)
df_arr = df.to_numpy() 

## Delete Table Value ###         
query_exec_id = wr.athena.start_query_execution(sql="DELETE FROM marketstatus.master_iceberg", 
                                                database='marketstatus', boto3_session=sess, wait=True)

## Data Insert To Iceberg Table using Batch###         
start_batch = 0
end_batch = 0
batch_size = 1500

while end_batch < len(df_arr):
    end_batch += batch_size
    if end_batch > len(df_arr):
        end_batch = len(df_arr) + 1    
    temp_arr = df_arr[start_batch:end_batch]
    query = """INSERT INTO marketstatus.master_iceberg VALUES """
    for r in temp_arr:
        query += f"('{r[0]}', '{r[1]}', '{r[2]}', '{r[3]}', '{r[4]}', '{r[5]}', '{r[6]}', '{r[7]}', '{r[8]}', '{r[9]}', '{r[10]}', '{r[11]}','{r[12]}','{r[13]}'),\n"
    query = query[:-2]
    query_exec_id = wr.athena.start_query_execution(sql=query, database='marketstatus', boto3_session=sess, wait=True)
    print(query_exec_id)

    start_batch += batch_size

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2518 entries, 0 to 2517
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Category            2518 non-null   object 
 1   Importer            2518 non-null   object 
 2   Brand               2518 non-null   object 
 3   Model               2518 non-null   object 
 4   Model (until 2021)  1571 non-null   object 
 5   Model group         2437 non-null   object 
 6   Body size           2512 non-null   object 
 7   Body style          2512 non-null   object 
 8   Fuel                2437 non-null   object 
 9   Brand segment       2437 non-null   object 
 10  CC                  1379 non-null   object 
 11  Price('000)         1520 non-null   float64
 12  LCV                 30 non-null     object 
 13  Commercial Type     75 non-null     object 
dtypes: float64(1), object(13)
memory usage: 275.5+ KB
None


AttributeError: Can only use .str accessor with string values!

In [15]:
df

,Category,Importer,Brand,Model,Model (until 2021),Model group,Body size,Body style,Fuel,Brand segment,CC,Price('000),LCV,Commercial Type
0,Passenger,BMW Group Korea,BMW,iX3,iX3,iX3,B,SUV,BEV,Premium,0,75900.0,NaN,NaN
1,Passenger,BMW Group Korea,BMW,iX xDrive50,iX xDrive50,iX,C,SUV,BEV,Premium,0,146300.0,NaN,NaN
2,Passenger,BMW Group Korea,BMW,iX xDrive40,iX xDrive40,iX,C,SUV,BEV,Premium,0,122600.0,NaN,NaN
3,Passenger,BMW Group Korea,BMW,i4 M50,i4 M50,i4,B,Sedan,BEV,Premium,0,84900.0,NaN,NaN
4,Passenger,BMW Group Korea,BMW,i4 eDrive40,i4 eDrive40,i4,B,Sedan,BEV,Premium,0,66500.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2513,Passenger,BMW Group Korea,BMW,iX1 xDrive30,iX1 xDrive30,iX1,A,SUV,BEV,Premium,NaN,NaN,NaN,NaN
2514,Passenger,Mercedes-Benz Korea,Mercedes-Benz,EQE 300,EQE 300,EQE,C,Sedan,BEV,Premium,NaN,NaN,NaN,NaN
2515,Passenger,Mercedes-Benz Korea,Mercedes-Benz,EQE 350 4MATIC,EQE 350 4MATIC,EQE,C,Sedan,BEV,Premium,NaN,NaN,NaN,NaN
2516,Passenger,Hyundai,Hyundai,KONA(SX2) 1.6 T,NaN,Kona,A0,SUV,Petrol,Mass,NaN,NaN,NaN,NaN


In [6]:
## Domestic Data Load
df = wr.s3.read_csv("s3://marketstatus-metas/master/")
sess = boto3.Session(region_name='us-east-1') 

## Change Data Type
df = df[['Category', 'Importer', 'Brand', 'Model', 'Model (until 2021)', 'Model group', 'Body size','Body style', 'Fuel', 'Brand segment', 'CC', "Price('000)"]].copy()
for i in list(df):
    df[i] = df[i].str.replace('  ', ' ')
    df[i] = df[i].str.rstrip()
    df[i] = df[i].str.lstrip()
df.fillna('-', inplace = True)
df_arr = df.to_numpy() 

## Delete Table Value ###         
query_exec_id = wr.athena.start_query_execution(sql="DELETE FROM marketstatus.master_iceberg", 
                                                database='marketstatus', boto3_session=sess, wait=True)


In [ ]:
## Data Insert To Iceberg Table using Batch###         
start_batch = 0
end_batch = 0
batch_size = 1500

while end_batch < len(df_arr):
    end_batch += batch_size
    if end_batch > len(df_arr):
        end_batch = len(df_arr) + 1    
    temp_arr = df_arr[start_batch:end_batch]
    query = """INSERT INTO marketstatus.master_iceberg VALUES """
    for r in temp_arr:
        query += f"('{r[0]}', '{r[1]}', '{r[2]}', '{r[3]}', '{r[4]}', '{r[5]}', '{r[6]}', '{r[7]}', '{r[8]}', '{r[9]}', '{r[10]}', '{r[11]}'),\n"
    query = query[:-2]
    query_exec_id = wr.athena.start_query_execution(sql=query, database='marketstatus', boto3_session=sess, wait=True)
    print(query_exec_id)

    start_batch += batch_size